# APPROACH 1 (STRUCTURE)

In [ ]:
import pandas as pd
import xpress as xp
import ast
from time import time

In [ ]:
xp.init('C:/xpressmp/bin/xpauth.xpr')
start_time = time()
# dataset loading
# my_channel_df = pd.read_csv('data/FIRST_WEEK_channel_A_schedule.csv', parse_dates=['Date-Time'])
# movie_db_df = pd.read_csv('data/movie_database_with_license_fee.csv', parse_dates=['release_date'])
# other_channels_0_df = pd.read_csv('data/FIRST_WEEK_channel_0_schedule.csv', parse_dates=['Date-Time'])
# other_channels_1_df = pd.read_csv('data/FIRST_WEEK_channel_1_schedule.csv', parse_dates=['Date-Time'])
# other_channels_2_df = pd.read_csv('data/FIRST_WEEK_channel_2_schedule.csv', parse_dates=['Date-Time'])
# conversion_rates_0_df = pd.read_csv('data/FIRST_WEEK_channel_0_conversion_rates.csv', parse_dates=['Date-Time'])
# conversion_rates_1_df = pd.read_csv('data/FIRST_WEEK_channel_1_conversion_rates.csv', parse_dates=['Date-Time'])
# conversion_rates_2_df = pd.read_csv('data/FIRST_WEEK_channel_2_conversion_rates.csv', parse_dates=['Date-Time'])

my_channel_df = pd.read_csv('data/AGGREGATE_FIRST_WEEK_channel_A_schedule.csv', parse_dates=['Date-Time'])
movie_db_df = pd.read_csv('data/movie_database_with_license_fee_1000.csv', parse_dates=['release_date'])
other_channels_0_df = pd.read_csv('data/AGGREGATE_FIRST_WEEK_channel_0_schedule.csv', parse_dates=['Date-Time'])
other_channels_1_df = pd.read_csv('data/AGGREGATE_FIRST_WEEK_channel_1_schedule.csv', parse_dates=['Date-Time'])
other_channels_2_df = pd.read_csv('data/AGGREGATE_FIRST_WEEK_channel_2_schedule.csv', parse_dates=['Date-Time'])
conversion_rates_0_df = pd.read_csv('data/AGGREGATE_FIRST_WEEK_channel_0_conversion_rates.csv', parse_dates=['Date-Time'])
conversion_rates_1_df = pd.read_csv('data/AGGREGATE_FIRST_WEEK_channel_1_conversion_rates.csv', parse_dates=['Date-Time'])
conversion_rates_2_df = pd.read_csv('data/AGGREGATE_FIRST_WEEK_channel_2_conversion_rates.csv', parse_dates=['Date-Time'])

# model initialization
model = xp.problem()
print('problem intialised at time ', time() - start_time)
# Decision Variables for Movie Scheduling
movie_indices = movie_db_df.index.tolist()
x = [[xp.var(name=f"x_{i}_{j}", vartype=xp.binary) for j in my_channel_df['Date-Time']] for i in movie_indices]
model.addVariable([var for sublist in x for var in sublist])  # Flatten and add all variables to the model

# Decision Variables for Advertising
ad_indices = ['Channel_0', 'Channel_1', 'Channel_2']
ad_vars = {ch: xp.var(name=f"ad_{ch}", vartype=xp.binary) for ch in ad_indices}
model.addVariable(list(ad_vars.values()))
# model.addVariable(ad_vars.values())  # Add advertising variables to the model
print('first set of desicion vars added at time ', time() - start_time)

# auxiliary binary variables to track genre presence per day
my_channel_df['Day'] = my_channel_df['Date-Time'].dt.date
genre_presence = {
    (day, genre): xp.var(vartype=xp.binary, name=f"genre_presence_{day}_{genre}")
    for day in my_channel_df['Day'].unique()
    for genre in movie_db_df['genres'].explode().unique()
}
print('desicion vars genre presence per day added at time ', time() - start_time)

# Add the auxiliary variables to the model
model.addVariable(list(genre_presence.values()))

# Objective Function: Maximization of the total viewership's revenue minus costs
viewership_from_movies = xp.Sum(x[i][j] * movie_db_df['scaled_popularity'].iloc[i] for i in movie_indices for j in range(len(my_channel_df)))
print('viewership_from_movies intialised at time ', time() - start_time)
# Create a mapping of conversion rates for each channel
conversion_rates_mapping = {
    'Channel_0': conversion_rates_0_df,
    'Channel_1': conversion_rates_1_df,
    'Channel_2': conversion_rates_2_df
}
movie_db_df['genres'] = movie_db_df['genres'].apply(ast.literal_eval)
# viewership_from_ads = xp.Sum(
#     ad_vars[ch] * xp.Sum(
#         x[i][j] * conversion_rates_mapping[ch].loc[j, genre]  # Usage of genre-based conversion rates
#         for j in range(len(my_channel_df))
#         for genre in list(movie_db_df.loc[i, 'genres'])  # Iterating through genres for each movie
#     )
#     for i in movie_indices for ch in ad_indices
# )
# print('viewrship_from_ads intialised at time ', time() - start_time)
# Total costs
license_fees = xp.Sum(x[i][j] * movie_db_df['license_fee'].iloc[i] for i in movie_indices for j in range(len(my_channel_df)))
print('license_fee intialised at time ', time() - start_time)
ad_costs = xp.Sum(ad_vars[ch] * (other_channels_0_df['ad_slot_price'].sum() if ch == 'Channel_0' else
                                  other_channels_1_df['ad_slot_price'].sum() if ch == 'Channel_1' else
                                  other_channels_2_df['ad_slot_price'].sum())
                  for ch in ad_indices)  # Add advertising costs
print('ad_cost intialised at time ', time() - start_time)

# Objective Function: Maximize total viewership minus costs
# model.setObjective(viewership_from_movies + viewership_from_ads - license_fees - ad_costs, sense=xp.maximize)
model.setObjective(viewership_from_movies - license_fees - ad_costs, sense=xp.maximize)
print('objective function set at time ', time() - start_time)

# Constraints

No_of_Time_slots = range(len(my_channel_df))

# 1. Time slot constraint: You can only schedule one movie per time slot
time_slots = my_channel_df['Date-Time'].unique()
model.addConstraint(xp.Sum(x[i][j] for i in movie_indices) == 1 for j in No_of_Time_slots)
print('constraint 1 added at time ', time() - start_time)

# 2. Movie must be scheduled for its whole time
# Ensure the movie is scheduled for its entire runtime if it is scheduled at time slot j
# STILL NEED TO LOOK AT THIS CAREFULLY
model.addConstraint(
    xp.Sum(x[i][j + k] for k in range(movie_db_df['runtime'].iloc[i] // 30) if j + k < len(my_channel_df)) == x[i][j] * movie_db_df['runtime'].iloc[i]
    for i in movie_indices for j in No_of_Time_slots
    )
print('constraint 2 added at time ', time() - start_time)

# 3. Total runtime constraint: Total scheduled runtime should not exceed a limit (24 hours for us)
max_runtime = 24 * 60  # in minutes
model.addConstraint(xp.Sum(x[i][j] * movie_db_df['runtime'].iloc[i] for i in movie_indices for j in No_of_Time_slots) <= max_runtime)
print('constraint 3 added at time ', time() - start_time)

# 4. Consecutive time slots constraint
for i in movie_indices:
    for j in range(len(my_channel_df)):
        for k in range(j + 1, len(my_channel_df)):
            final_time = my_channel_df['Date-Time'].iloc[k]
            initial_time = my_channel_df['Date-Time'].iloc[j]
            slot_duration = (final_time - initial_time).total_seconds() / 60
            model.addConstraint(x[i][j] * slot_duration <= x[i][j] * movie_db_df['runtime'].iloc[i])
print('constraint 4 added at time ', time() - start_time)

# 5. Budget constraint for movies
total_budget = 1000000  # Example budget
model.addConstraint(
    xp.Sum(x[i][j] * movie_db_df['budget'].iloc[i] for i in movie_indices for j in range(len(my_channel_df))) <= total_budget
)
print('constraint 5 added at time ', time() - start_time)

# 6. Advertising budget constraint
total_ad_budget = 50000  # Example advertising budget
model.addConstraint(
    xp.Sum(ad_vars[ch] * (other_channels_0_df['ad_slot_price'].sum() if ch == 'Channel_0' else
                          other_channels_1_df['ad_slot_price'].sum() if ch == 'Channel_1' else
                          other_channels_2_df['ad_slot_price'].sum())
               for ch in ad_indices) <= total_ad_budget
)
print('constraint 6 added at time ', time() - start_time)

# 7. Threshold for Conversion Rates

# need new decision variable for movie i advertised pon channel c at slot t

conversion_rate_threshold = 0.2  # Example threshold
for i in movie_indices:
    for j in No_of_Time_slots:
        for ch in ad_indices:
            # Single check: Ensure j is within bounds for conversion_rates_mapping[ch] and genre exists
            genre = movie_db_df['genres'].iloc[i][0]  # Assume the first genre is primary

            if j < len(conversion_rates_mapping[ch]) and genre in conversion_rates_mapping[ch].columns:
                model.addConstraint(
                    ad_vars[ch] * x[i][j] * conversion_rates_mapping[ch].iloc[j][genre] >=
                    ad_vars[ch] * x[i][j] * conversion_rate_threshold
                )

print('constraint 7 added at time ', time() - start_time)

# 8. Daily Genre Diversity Constraint
max_genres_per_day = 3   # maximum number of genres allowed per day

# genre presence constraints and diversity constraint
for day in my_channel_df['Day'].unique():
    # Get all time slots for the current day
    daily_slots = my_channel_df[my_channel_df['Day'] == day].index.tolist()
    
    for genre in movie_db_df['genres'].explode().unique():
        # Check if (day, genre) is a valid key in genre_presence
        if (day, genre) in genre_presence:
            # Enforce that genre_presence[day, genre] is set to 1 if the genre is scheduled at least once on this day
            model.addConstraint(
                xp.Sum(
                    x[i][j] for i in movie_indices for j in daily_slots if genre in movie_db_df['genres'].iloc[i]
                ) >= genre_presence[day, genre]
            )
    
    # Total genres per day should not exceed max_genres_per_day
    model.addConstraint(
        xp.Sum(genre_presence[day, genre] for genre in movie_db_df['genres'].explode().unique() 
               if (day, genre) in genre_presence) <= max_genres_per_day
    )

print('Constraint 8: Daily genre diversity constraint added at time :', time() - start_time)

# # 9. Genre Clashes Constraint
# competitor_schedules = pd.concat([
#     other_channels_0_df[['Date-Time', 'content_type']],
#     other_channels_1_df[['Date-Time', 'content_type']],
#     other_channels_2_df[['Date-Time', 'content_type']]
# ], ignore_index=True)

# # mapping from movie titles to genres in the movie database
# title_to_genre = {row['title']: row['genres'] for index, row in movie_db_df.iterrows()}

# # genre info added to the competitor schedules based on titles
# competitor_schedules['genre'] = competitor_schedules['content_type'].map(title_to_genre)

# # keep only the scheduled movies (not advertisements)
# competitor_movies = competitor_schedules[competitor_schedules['content_type'] == 'Movie']

# # constraints to avoid genre clashes
# for i in movie_indices:
#     movie_title = movie_db_df['title'].iloc[i]  # title of the movie to be scheduled
#     movie_genres = set(title_to_genre[movie_title])  # retrieve genres of the movie and convert to a set

#     for j in range(len(my_channel_df)):
#         if my_channel_df['Date-Time'].iloc[j] is not None:
#             # Find all competitor movies scheduled at the same time
#             competing_movies = competitor_movies[competitor_movies['Date-Time'] == my_channel_df['Date-Time'].iloc[j]]
            
#             # now, constraints to limit genre clashes
#             for _, competing_movie in competing_movies.iterrows():
#                 competing_movie_genres = set(competing_movie['genre'])  # Get genres of the competing movie
                
#                 # Check for genre clash using intersection
#                 if movie_genres.intersection(competing_movie_genres):
#                     model.addConstraint(x[i][j] + xp.Sum(x[other_i][j] for other_i in movie_indices 
#                                                           if set(title_to_genre[movie_db_df['title'].iloc[other_i]]).intersection(competing_movie_genres)) > 0 <= 1)

# print('Constraint 9: Genre clashes constraint added at time ', time() - start_time)
print('Constraint 9: Genre clashes TO DO AFTER INTEGRATING 1ST WEEK DATASET')

# Solve the model
model.solve()
print('model solved at time ', time() - start_time)

# Output the results for scheduled movies
for i in movie_indices:
    for j in range(len(my_channel_df)):
        if model.getSolution(x[i][j]) > 0:  # Movie is scheduled
            scheduled_time = my_channel_df['Date-Time'].iloc[j]
            print(f"Scheduled Movie: {movie_db_df['title'].iloc[i]}, Time Slot: {scheduled_time}")

# Output the results for advertising
for ch in ad_indices:
    if model.getSolution(ad_vars[ch]) > 0:
        print(f"Advertising on {ch}")

# Optionally, display the objective value
print("Maximized Viewership:", model.getObjVal())

# APPROACH 2 : WORKING CODE 1

In [4]:
import pandas as pd
import numpy as np
import xpress as xp
from datetime import datetime, timedelta
from time import time
import os

from time_slot_viewership import movie_views_for_time_slot,comp_advertised_views_for_time_slot, own_advertised_views_for_time_slot, calculate_ad_slot_price

xp.init('C:/xpressmp/bin/xpauth.xpr')

my_channel_df = pd.read_csv('data/AGGREGATE_FIRST_WEEK_channel_A_schedule.csv', parse_dates=['Date-Time'])
movie_db_df = pd.read_csv('data/movie_database_with_license_fee_100.csv', parse_dates=['release_date'])
movie_genre_df = pd.read_csv('data/movie_genre_hot_one_100.csv')


channel_0_df = pd.read_csv('data/AGGREGATE_FIRST_WEEK_channel_0_schedule.csv', parse_dates=['Date-Time'])
channel_1_df = pd.read_csv('data/AGGREGATE_FIRST_WEEK_channel_1_schedule.csv', parse_dates=['Date-Time'])
channel_2_df = pd.read_csv('data/AGGREGATE_FIRST_WEEK_channel_2_schedule.csv', parse_dates=['Date-Time'])
conversion_rates_0_df = pd.read_csv('data/AGGREGATE_FIRST_WEEK_channel_0_conversion_rates.csv', parse_dates=['Date-Time'])
conversion_rates_1_df = pd.read_csv('data/AGGREGATE_FIRST_WEEK_channel_1_conversion_rates.csv', parse_dates=['Date-Time'])
conversion_rates_2_df = pd.read_csv('data/AGGREGATE_FIRST_WEEK_channel_2_conversion_rates.csv', parse_dates=['Date-Time'])


first_week_cutoff = datetime(2024, 10, 2, 23, 59, 0)
my_channel_df = my_channel_df.drop(my_channel_df[my_channel_df['Date-Time'] > first_week_cutoff].index)

model = xp.problem()

Demos = ['children', 'adults', 'retirees']
Genres = movie_genre_df.columns.to_list()

number_of_movies = len(movie_db_df.index)
number_of_time_slots = len(my_channel_df.index)
number_of_comp_channels = 3

Movies = range(number_of_movies)
Time_slots = range(number_of_time_slots)
Channels = range(number_of_comp_channels)

channel_dict = {
    0: channel_0_df,
    1: channel_1_df,
    2: channel_2_df
}

conversion_dict = {
    0: conversion_rates_0_df,
    1: conversion_rates_1_df,
    2: conversion_rates_2_df
}

population = 1000000
viewership_units = 1000
ad_sell_price_per_unit = 100
budget = 1000000

##########################
# Decision Variables
##########################

# whether to schedule movie i at time slot j
x = np.array(
    [xp.var(name=f"x_{i}_{j}", vartype=xp.binary) for i in Movies for j in Time_slots],
    dtype=xp.npvar
).reshape(number_of_movies,number_of_time_slots)
model.addVariable(x)

# whether to show movie i
y = [xp.var(name=f"y_{i}", vartype=xp.binary) for i in Movies]
model.addVariable(y)

# whether movie i is advertises on channel c at time slot j
z = np.array(
    [xp.var(name=f'z_{i}_{j}_{c}', vartype=xp.binary) for i in Movies for j in Time_slots for c in Channels],
    dtype=xp.npvar
).reshape(number_of_movies,number_of_time_slots, number_of_comp_channels)
model.addVariable(z)

# whether movie i is advertised on our own channel at time slot j
w = np.array(
    [xp.var(name=f"w_{i}_{j}", vartype=xp.binary) for i in Movies for j in Time_slots],
    dtype=xp.npvar
).reshape(number_of_movies,number_of_time_slots)
model.addVariable(w)

# whether advert slot at time slot j is sold
v = [xp.var(name=f"v_{j}", vartype=xp.binary) for j in Time_slots]
model.addVariable(v)

# 1000 viewership, used to calulate value of ad slot j to sell
u = [xp.var(name=f"u_{j}", vartype=xp.integer) for j in Time_slots]
model.addVariable(u)

# start time movie i
s = np.array([xp.var( name='s_{0}'.format(i), vartype=xp.continuous)
                    for i in Movies], dtype=xp.npvar).reshape(number_of_movies)
model.addVariable(s)

# end time movie i
e = np.array([xp.var( name='e_{0}'.format(i), vartype=xp.continuous)
                    for i in Movies], dtype=xp.npvar).reshape(number_of_movies)
model.addVariable(e)

##########################
# Constraints
##########################

# 1. You can only schedule one movie per time slot
model.addConstraint(xp.Sum(x[i][j] for i in Movies) == 1 for j in Time_slots)

# 2. Movie must be scheduled for whole length
model.addConstraint(30*xp.Sum(x[i][j] for j in Time_slots) == y[i]*movie_db_df['runtime_with_ads'].loc[i] for i in Movies)

# 3. Start and end time must be length of movie
model.addConstraint(e[i] - s[i] == y[i]*movie_db_df['runtime_with_ads'].loc[i] for i in Movies)

# 4. Movie must be scheduled for consecutive time slots
start_of_week = datetime(2024, 10, 1, 0, 0, 0)

model.addConstraint(
    e[i] >= x[i][j]*(my_channel_df['Date-Time'].loc[j] + timedelta(minutes=30)- start_of_week).total_seconds()/60
    for i in Movies for j in Time_slots
    )

model.addConstraint(
    s[i] <= x[i][j]*(my_channel_df['Date-Time'].loc[j] - start_of_week).total_seconds()/60 + (1 - x[i][j])*(2880 - movie_db_df['runtime_with_ads'].loc[i])
    for i in Movies for j in Time_slots
    )

# 5. Only one ad can be bought per avialable slot
model.addConstraint(
    xp.Sum(z[i][j][c] for i in Movies) <= 1 for j in Time_slots for c in Channels
)

model.addConstraint(
    xp.Sum(w[i][j] for i in Movies) + v[j] <= 1 for j in Time_slots
)

# 6. Only advertise movie if it is shown
model.addConstraint(
    z[i][j][c] <= y[i] for i in Movies for j in Time_slots for c in Channels
)
model.addConstraint(
    w[i][j] <= y[i] for i in Movies for j in Time_slots
)

# 7. Only advertise before the movie is scheduled
model.addConstraint(
    z[i][j][c]*(my_channel_df['Date-Time'].loc[j] - start_of_week + timedelta(minutes=30)).total_seconds()/60 <= s[i]
    for i in Movies for j in Time_slots for c in Channels
)
model.addConstraint(
    w[i][j]*(my_channel_df['Date-Time'].loc[j] - start_of_week + timedelta(minutes=30)).total_seconds()/60 <= s[i]
    for i in Movies for j in Time_slots
)

# 8. The number per thousand of viewership is less than the viewership for the time slot
model.addConstraint(
    u[j]*viewership_units <= 
    xp.Sum(
        movie_views_for_time_slot(x, i, j, movie_db_df, my_channel_df, Demos, population)
        for i in Movies
        )
    + xp.Sum(
        comp_advertised_views_for_time_slot(z, i, j, c, movie_db_df, channel_dict, conversion_dict, Demos, Genres, population)
        for i in Movies for c in Channels
        )
    + xp.Sum(
        own_advertised_views_for_time_slot(w, i , j, movie_db_df, my_channel_df, Demos, population)
        for i in Movies
    )
    for j in Time_slots
)

#9. Ad slot is only sold if own movie is not advertised at time slot j
model.addConstraint(
    u[j] <= v[j]*(population/viewership_units)
    for j in Time_slots
)

# 10. license fees and advertising slots bought must be within budget
# model.addConstraint(
#     xp.Sum(
#         x[i][j] * movie_db_df['license_fee'].iloc[i]
#         for i in Movies for j in Time_slots
#     )
#     + xp.Sum(
#         z[i][j][c] * calculate_ad_slot_price(j, channel_dict[c])
#         for i in Movies for j in Time_slots for c in Channels
#     )
#     <= budget
# )

##########################
# Objective Function
##########################

model.setObjective(
    xp.Sum(ad_sell_price_per_unit*u[j] for j in Time_slots),
    sense=xp.maximize
)

# model.controls.maxtime = 300
model.controls.maxnode = 1000  # Limit to 1000 nodes
model.controls.miprelstop = 0.01  # Stop if relative gap is less than 1%
# model.controls.tunermaxtime = 1000
# model.controls.timelimit = 60
# model.tune('g')

solvestatus, solstatus = model.optimize()

now = datetime.now()
now = str(now).replace(" ", "_")
now = now.replace(":", "-")

saved_sol_path = f'solutions/scheduling_advert_demos_{now}'
model.write(saved_sol_path)

cost = sum(model.getSolution(x[i][j]) * movie_db_df['license_fee'].iloc[i] for i in Movies for j in Time_slots) + sum(model.getSolution(z[i][j][c]) * calculate_ad_slot_price(j, channel_dict[c]) for i in Movies for j in Time_slots for c in Channels)
print(cost)

output_dir = "./output"
# create the dir if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

if solstatus != xp.SolStatus.INFEASIBLE or solstatus != xp.SolStatus.UNBOUNDED or solstatus != xp.SolStatus.UNBOUNDED:
    with open(f"./output/output_{str(now)}.txt", "w") as f:
        f.write('Viewership: ')
        f.write(str(model.getObjVal))
        f.write('\n')
        for j in Time_slots:
            for i in Movies:
                if model.getSolution(x[i][j]) == 1:
                    f.write("At ")
                    f.write(str(my_channel_df['Date-Time'].loc[j]))
                    f.write(" show movie ")
                    f.write(movie_db_df['title'].loc[i])
                    f.write('\n')
    f.close()


FICO Xpress v9.4.2, Hyper, solve started 22:42:35, Nov 12, 2024
Heap usage: 32MB (peak 32MB, 4863KB system)
Maximizing MILP noname using up to 8 threads and up to 15GB memory, with these control settings:
MAXNODE = 1000
OUTPUTLOG = 1
MIPRELSTOP = .01
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
     69683 rows        34938 cols       214476 elements     34738 entities
Presolved problem has:
     53242 rows        26900 cols       165857 elements     26700 entities
LP relaxation tightened
Presolve finished in 1 seconds
Heap usage: 49MB (peak 75MB, 4863KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 8.89e-09,  1.63e+05] / [ 2.48e-13,  1.99e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  2.79e+03] / [ 1.00e+00,  1.00e+03]
  Objective      [min,max] : [ 1.00e+02,  1.00e+02] / [ 1.00e+02,  1.00e+02]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory u

# With ADVERTS now

In [14]:
import pandas as pd
import numpy as np
import xpress as xp
from datetime import datetime, timedelta
from time import time
import os

from time_slot_viewership import movie_views_for_time_slot,comp_advertised_views_for_time_slot, own_advertised_views_for_time_slot, calculate_ad_slot_price

start_time = time()
time_stamp = 0

xp.init('C:/xpressmp/bin/xpauth.xpr')

my_channel_df = pd.read_csv('data/AGGREGATE_FIRST_WEEK_channel_A_schedule.csv', parse_dates=['Date-Time'])
movie_db_df = pd.read_csv('data/movie_database_with_license_fee_100.csv', parse_dates=['release_date'])
movie_genre_df = pd.read_csv('data/movie_genre_hot_one_100.csv')

channel_0_df = pd.read_csv('data/ADVERTS_FIRST_WEEK_channel_0_schedule.csv', parse_dates=['Date-Time'])
channel_1_df = pd.read_csv('data/ADVERTS_FIRST_WEEK_channel_1_schedule.csv', parse_dates=['Date-Time'])
channel_2_df = pd.read_csv('data/ADVERTS_FIRST_WEEK_channel_2_schedule.csv', parse_dates=['Date-Time'])
conversion_rates_0_df = pd.read_csv('data/FIRST_WEEK_channel_0_conversion_rates.csv', parse_dates=['Date-Time'])
conversion_rates_1_df = pd.read_csv('data/FIRST_WEEK_channel_1_conversion_rates.csv', parse_dates=['Date-Time'])
conversion_rates_2_df = pd.read_csv('data/FIRST_WEEK_channel_2_conversion_rates.csv', parse_dates=['Date-Time'])


first_week_cutoff = datetime(2024, 10, 2, 23, 59, 0)
my_channel_df = my_channel_df.drop(my_channel_df[my_channel_df['Date-Time'] > first_week_cutoff].index)

model = xp.problem()

Demos = ['children', 'adults', 'retirees']
Genres = movie_genre_df.columns.to_list()

number_of_movies = len(movie_db_df.index)
number_of_time_slots = len(my_channel_df.index)
number_of_comp_channels = 3
number_of_ad_slots_0 = len(conversion_rates_0_df)
number_of_ad_slots_1 = len(conversion_rates_1_df)
number_of_ad_slots_2 = len(conversion_rates_2_df)

Movies = range(number_of_movies)
Time_slots = range(number_of_time_slots)
Channels = range(number_of_comp_channels)

Ad_slots_0 = range(number_of_ad_slots_0)
Ad_slots_1 = range(number_of_ad_slots_1)
Ad_slots_2 = range(number_of_ad_slots_2)

channel_dict = {
    0: channel_0_df,
    1: channel_1_df,
    2: channel_2_df
}

conversion_dict = {
    0: conversion_rates_0_df,
    1: conversion_rates_1_df,
    2: conversion_rates_2_df
}

population = 1000000
viewership_units = 1000
ad_sell_price_per_unit = 100
budget = 1000000

##########################
# Decision Variables
##########################

# whether to schedule movie i at time slot j
x = np.array(
    [xp.var(name=f"x_{i}_{j}", vartype=xp.binary) for i in Movies for j in Time_slots],
    dtype=xp.npvar
).reshape(number_of_movies,number_of_time_slots)
model.addVariable(x)

# whether to show movie i
y = [xp.var(name=f"y_{i}", vartype=xp.binary) for i in Movies]
model.addVariable(y)

# whether movie i is advertises on channel 0 at time slot j
z0 = np.array(
    [xp.var(name=f'z0_{i}_{r}', vartype=xp.binary) for i in Movies for r in Ad_slots_0],
    dtype=xp.npvar
).reshape(number_of_movies,number_of_ad_slots_0)
model.addVariable(z0)

# whether movie i is advertises on channel 0 at time slot j
z1 = np.array(
    [xp.var(name=f'z1_{i}_{s}', vartype=xp.binary) for i in Movies for s in Ad_slots_1],
    dtype=xp.npvar
).reshape(number_of_movies,number_of_ad_slots_1)
model.addVariable(z1)

# whether movie i is advertises on channel 0 at time slot j
z2 = np.array(
    [xp.var(name=f'z2_{i}_{t}', vartype=xp.binary) for i in Movies for t in Ad_slots_2],
    dtype=xp.npvar
).reshape(number_of_movies,number_of_ad_slots_2)
model.addVariable(z2)

# whether movie i is advertised on our own channel at time slot j
w = np.array(
    [xp.var(name=f"w_{i}_{j}", vartype=xp.binary) for i in Movies for j in Time_slots],
    dtype=xp.npvar
).reshape(number_of_movies,number_of_time_slots)
model.addVariable(w)

# whether advert slot at time slot j is sold
# v = np.array(
#     [xp.var(name=f"v_{i}_{j}", vartype=xp.binary) for i in Movies for j in Time_slots]
# ).reshape(number_of_movies,number_of_time_slots)
# model.addVariable(v)

# sum of viewers for movie i across time slots shown 
u = [xp.var(name=f"u_{i}", vartype=xp.integer) for i in Movies]
model.addVariable(u)

# start time movie i
start = np.array([xp.var( name='s_{0}'.format(i), vartype=xp.continuous)
                    for i in Movies], dtype=xp.npvar).reshape(number_of_movies)
model.addVariable(start)

# end time movie i
end = np.array([xp.var( name='e_{0}'.format(i), vartype=xp.continuous)
                    for i in Movies], dtype=xp.npvar).reshape(number_of_movies)
model.addVariable(end)

##########################
# Constraints
##########################

# 1. You can only schedule one movie per time slot
model.addConstraint(xp.Sum(x[i][j] for i in Movies) == 1 for j in Time_slots)

# 2. Movie must be scheduled for whole length
model.addConstraint(30*xp.Sum(x[i][j] for j in Time_slots) == y[i]*movie_db_df['runtime_with_ads'].loc[i] for i in Movies)

# 3. Start and end time must be length of movie
model.addConstraint(end[i] - start[i] == y[i]*movie_db_df['runtime_with_ads'].loc[i] for i in Movies)

# 4. Movie must be scheduled for consecutive time slots
start_of_week = datetime(2024, 10, 1, 0, 0, 0)

model.addConstraint(
    end[i] >= x[i][j]*(my_channel_df['Date-Time'].loc[j] + timedelta(minutes=30)- start_of_week).total_seconds()/60
    for i in Movies for j in Time_slots
    )

model.addConstraint(
    start[i] <= x[i][j]*(my_channel_df['Date-Time'].loc[j] - start_of_week).total_seconds()/60 + (1 - x[i][j])*(2880 - movie_db_df['runtime_with_ads'].loc[i])
    for i in Movies for j in Time_slots
    )

print('Scheduling constaints added, ', time() - start_time)
# 5. Only one ad can be bought per avialable slot
model.addConstraint(
    xp.Sum(z0[i][r] for i in Movies) <= 1 for r in Ad_slots_0
)

model.addConstraint(
    xp.Sum(z1[i][s] for i in Movies) <= 1 for s in Ad_slots_1
)

model.addConstraint(
    xp.Sum(z2[i][t] for i in Movies) <= 1 for t in Ad_slots_2
)

# model.addConstraint(
#     xp.Sum(w[i][j] for i in Movies) + xp.Sum(v[i][j] for i in Movies) == 1 for j in Time_slots
# )


# 6. Only advertise movie if it is shown
model.addConstraint(
    z0[i][r] <= y[i] for i in Movies for r in Ad_slots_0
)
model.addConstraint(
    z1[i][s] <= y[i] for i in Movies for s in Ad_slots_1
)
model.addConstraint(
    z2[i][t] <= y[i] for i in Movies for t in Ad_slots_2
)
# model.addConstraint(
#     w[i][j] <= y[i] for i in Movies for j in Time_slots
# )

# # 7. Only sell ad slot for movie i at time j if it is shown then
# model.addConstraint(
#     v[i][j] <= x[i][j] for i in Movies for j in Time_slots
# )

# 8. Only advertise before the movie is scheduled
model.addConstraint(
    z0[i][r]*(conversion_rates_0_df['Date-Time'].loc[r] - start_of_week + timedelta(minutes=30)).total_seconds()/60 <= start[i]
    for i in Movies for r in Ad_slots_0
)
model.addConstraint(
    z1[i][s]*(conversion_rates_1_df['Date-Time'].loc[s] - start_of_week + timedelta(minutes=30)).total_seconds()/60 <= start[i]
    for i in Movies for s in Ad_slots_1
)
model.addConstraint(
    z2[i][t]*(conversion_rates_2_df['Date-Time'].loc[t] - start_of_week + timedelta(minutes=30)).total_seconds()/60 <= start[i]
    for i in Movies for t in Ad_slots_2
)
# model.addConstraint(
#     w[i][j]*(my_channel_df['Date-Time'].loc[j] - start_of_week + timedelta(minutes=30)).total_seconds()/60 <= start[i]
#     for i in Movies for j in Time_slots
# )

print('Advertising constraints added, ', time() - start_time)
# 9. The number per thousand of viewership is less than the viewership for the time slot
model.addConstraint(
    u[i]*viewership_units <=
    xp.Sum(
        movie_views_for_time_slot(x, i, j, movie_db_df, my_channel_df, Demos, population)
        for j in Time_slots
    )
    # + xp.Sum(
    #     own_advertised_views_for_time_slot(w, i , j, movie_db_df, my_channel_df, Demos, population)
    #     for j in Time_slots
    # )
    + xp.Sum(
        # comp_advertised_views_for_time_slot(z, i, j, c, movie_db_df, channel_dict, conversion_dict, Demos, Genres, population)
        comp_advertised_views_for_time_slot(z0, i, r, movie_db_df, channel_0_df, conversion_rates_0_df, Demos, Genres, population)
        for r in Ad_slots_0
    )
    + xp.Sum(
        comp_advertised_views_for_time_slot(z1, i, s, movie_db_df, channel_0_df, conversion_rates_0_df, Demos, Genres, population)
        for s in Ad_slots_1
    )
    + 
    xp.Sum(
        comp_advertised_views_for_time_slot(z2, i, t, movie_db_df, channel_0_df, conversion_rates_0_df, Demos, Genres, population)
        for t in Ad_slots_2
    )
    for i in Movies
)


# # 12. We only get contribution for viewership for movie i at time slot j if the time slot is sold
# model.addConstraint(
#     u[i] <= v[i][j]*(population/viewership_units)
#     for i in Movies for j in Time_slots
# )


print('Viewership constaints added, ', time() - start_time)

# 12. license fees and advertising slots bought must be within budget
# model.addConstraint(
#     xp.Sum(
#         y[i] * movie_db_df['license_fee'].iloc[i]
#         for i in Movies
#     )
#     + xp.Sum(
#         z[i][j][c] * calculate_ad_slot_price(j, channel_dict[c])
#         for i in Movies for j in Time_slots for c in Channels
#     )
#     <= budget
# )


##########################
# Objective Function
##########################

model.setObjective(
    xp.Sum(ad_sell_price_per_unit*u[i]  for i in Movies),
    sense=xp.maximize
)

print('time to intialise problem: ', time() - start_time)

# model.controls.maxtime = 300
# model.controls.maxnode = 1000
model.controls.miprelstop = 0.01
# model.controls.tunermaxtime = 1000
# model.controls.timelimit = 60
# model.tune('g')

solvestatus, solstatus = model.optimize()

now = datetime.now()
now = str(now).replace(" ", "_")
now = now.replace(":", "-")

# saved_sol_path = f'solutions/scheduling_advert_demos_{now}'
# model.write(saved_sol_path)

cost = sum(model.getSolution(y[i]) * movie_db_df['license_fee'].iloc[i] for i in Movies)
+ sum(model.getSolution(z0[i][j]) * calculate_ad_slot_price(j, channel_0_df) for i in Movies for j in Ad_slots_0)
+ sum(model.getSolution(z1[i][j]) * calculate_ad_slot_price(j, channel_1_df) for i in Movies for j in Ad_slots_1)
+ sum(model.getSolution(z2[i][j]) * calculate_ad_slot_price(j, channel_2_df) for i in Movies for j in Ad_slots_2)
print(cost)
# if solstatus != xp.SolStatus.INFEASIBLE or solstatus != xp.SolStatus.UNBOUNDED or solstatus != xp.SolStatus.UNBOUNDED:

output_dir = "./output"
# create the dir if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

with open(f"./output/output_{str(now)}.txt", "w") as f:
    f.write('Viewership: ')
    f.write(str(model.getObjVal))
    f.write('\n')
    for j in Time_slots:
        for i in Movies:
            if model.getSolution(x[i][j]) == 1:
                f.write("At ")
                f.write(str(my_channel_df['Date-Time'].loc[j]))
                f.write(" show movie ")
                f.write(movie_db_df['title'].loc[i])
                f.write('\n')
    # for j in Time_slots:
    #     for i in Movies:
    #         if model.getSolution(w[i][j]) == 1:
    #             f.write("At ")
    #             f.write(str(my_channel_df['Date-Time'].loc[j]))
    #             f.write(" on own channel advertise movie ")
    #             f.write(movie_db_df['title'].loc[i])
    #             f.write('\n')
    for j in Ad_slots_0:
        for i in Movies:
            if model.getSolution(z0[i][j]) == 1:
                f.write("At ")
                f.write(str(channel_0_df['Date-Time'].loc[j]))
                f.write(" on channel 0 advertise movie ")
                f.write(movie_db_df['title'].loc[i])
                f.write('\n')
    for j in Ad_slots_1:
        for i in Movies:
            if model.getSolution(z1[i][j]) == 1:
                f.write("At ")
                f.write(str(channel_1_df['Date-Time'].loc[j]))
                f.write(" on channel 1 advertise movie ")
                f.write(movie_db_df['title'].loc[i])
                f.write('\n')
    for j in Ad_slots_2:
        for i in Movies:
            if model.getSolution(z2[i][j]) == 1:
                f.write("At ")
                f.write(str(channel_2_df['Date-Time'].loc[j]))
                f.write(" on channel 2 advertise movie ")
                f.write(movie_db_df['title'].loc[i])
                f.write('\n')
f.close()


Scheduling constaints added,  1.5184595584869385
Advertising constraints added,  5.309004783630371


TypeError: comp_advertised_views_for_time_slot() missing 1 required positional argument: 'population'

# A TEST CONSIDERING VIEW COUNT

In [ ]:
import pandas as pd
import numpy as np
import xpress as xp
from datetime import datetime, timedelta
from time import time
import os

from time_slot_viewership import movie_views_for_time_slot,comp_advertised_views_for_time_slot, own_advertised_views_for_time_slot, calculate_ad_slot_price

start_time = time()
time_stamp = 0

xp.init('C:/xpressmp/bin/xpauth.xpr')

my_channel_df = pd.read_csv('data/AGGREGATE_FIRST_WEEK_channel_A_schedule.csv', parse_dates=['Date-Time'])
movie_db_df = pd.read_csv('data/movie_database_with_license_fee_100.csv', parse_dates=['release_date'])
movie_genre_df = pd.read_csv('data/movie_genre_hot_one_100.csv')

channel_0_df = pd.read_csv('data/ADVERTS_FIRST_WEEK_channel_0_schedule.csv', parse_dates=['Date-Time'])
channel_1_df = pd.read_csv('data/ADVERTS_FIRST_WEEK_channel_1_schedule.csv', parse_dates=['Date-Time'])
channel_2_df = pd.read_csv('data/ADVERTS_FIRST_WEEK_channel_2_schedule.csv', parse_dates=['Date-Time'])
conversion_rates_0_df = pd.read_csv('data/FIRST_WEEK_channel_0_conversion_rates.csv', parse_dates=['Date-Time'])
conversion_rates_1_df = pd.read_csv('data/FIRST_WEEK_channel_1_conversion_rates.csv', parse_dates=['Date-Time'])
conversion_rates_2_df = pd.read_csv('data/FIRST_WEEK_channel_2_conversion_rates.csv', parse_dates=['Date-Time'])


first_week_cutoff = datetime(2024, 10, 2, 23, 59, 0)
my_channel_df = my_channel_df.drop(my_channel_df[my_channel_df['Date-Time'] > first_week_cutoff].index)

model = xp.problem()

Demos = ['children', 'adults', 'retirees']
Genres = movie_genre_df.columns.to_list()

number_of_movies = len(movie_db_df.index)
number_of_time_slots = len(my_channel_df.index)
number_of_comp_channels = 3
number_of_ad_slots_0 = len(conversion_rates_0_df)
number_of_ad_slots_1 = len(conversion_rates_1_df)
number_of_ad_slots_2 = len(conversion_rates_2_df)

Movies = range(number_of_movies)
Time_slots = range(number_of_time_slots)
Channels = range(number_of_comp_channels)

Ad_slots_0 = range(number_of_ad_slots_0)
Ad_slots_1 = range(number_of_ad_slots_1)
Ad_slots_2 = range(number_of_ad_slots_2)

channel_dict = {
    0: channel_0_df,
    1: channel_1_df,
    2: channel_2_df
}

conversion_dict = {
    0: conversion_rates_0_df,
    1: conversion_rates_1_df,
    2: conversion_rates_2_df
}

population = 1000000
viewership_units = 1000
ad_sell_price_per_unit = 100
budget = 1000000

##########################
# Decision Variables
##########################

# whether to schedule movie i at time slot j
x = np.array(
    [xp.var(name=f"x_{i}_{j}", vartype=xp.binary) for i in Movies for j in Time_slots],
    dtype=xp.npvar
).reshape(number_of_movies,number_of_time_slots)
model.addVariable(x)

# whether to show movie i
y = [xp.var(name=f"y_{i}", vartype=xp.binary) for i in Movies]
model.addVariable(y)

# whether movie i is advertises on channel 0 at time slot j
z0 = np.array(
    [xp.var(name=f'z0_{i}_{r}', vartype=xp.binary) for i in Movies for r in Ad_slots_0],
    dtype=xp.npvar
).reshape(number_of_movies,number_of_ad_slots_0)
model.addVariable(z0)

# whether movie i is advertises on channel 0 at time slot j
z1 = np.array(
    [xp.var(name=f'z1_{i}_{s}', vartype=xp.binary) for i in Movies for s in Ad_slots_1],
    dtype=xp.npvar
).reshape(number_of_movies,number_of_ad_slots_1)
model.addVariable(z1)

# whether movie i is advertises on channel 0 at time slot j
z2 = np.array(
    [xp.var(name=f'z2_{i}_{t}', vartype=xp.binary) for i in Movies for t in Ad_slots_2],
    dtype=xp.npvar
).reshape(number_of_movies,number_of_ad_slots_2)
model.addVariable(z2)

# whether movie i is advertised on our own channel at time slot j
w = np.array(
    [xp.var(name=f"w_{i}_{j}", vartype=xp.binary) for i in Movies for j in Time_slots],
    dtype=xp.npvar
).reshape(number_of_movies,number_of_time_slots)
model.addVariable(w)

# whether advert slot at time slot j is sold
# v = np.array(
#     [xp.var(name=f"v_{i}_{j}", vartype=xp.binary) for i in Movies for j in Time_slots]
# ).reshape(number_of_movies,number_of_time_slots)
# model.addVariable(v)

# sum of viewers for movie i across time slots shown 
u = [xp.var(name=f"u_{i}", vartype=xp.integer) for i in Movies]
model.addVariable(u)

# start time movie i
start = np.array([xp.var( name='s_{0}'.format(i), vartype=xp.continuous)
                    for i in Movies], dtype=xp.npvar).reshape(number_of_movies)
model.addVariable(start)

# end time movie i
end = np.array([xp.var( name='e_{0}'.format(i), vartype=xp.continuous)
                    for i in Movies], dtype=xp.npvar).reshape(number_of_movies)
model.addVariable(end)

##########################
# Constraints
##########################

# 1. You can only schedule one movie per time slot
model.addConstraint(xp.Sum(x[i][j] for i in Movies) == 1 for j in Time_slots)

# 2. Movie must be scheduled for whole length
model.addConstraint(30*xp.Sum(x[i][j] for j in Time_slots) == y[i]*movie_db_df['runtime_with_ads'].loc[i] for i in Movies)

# 3. Start and end time must be length of movie
model.addConstraint(end[i] - start[i] == y[i]*movie_db_df['runtime_with_ads'].loc[i] for i in Movies)

# 4. Movie must be scheduled for consecutive time slots
start_of_week = datetime(2024, 10, 1, 0, 0, 0)

model.addConstraint(
    end[i] >= x[i][j]*(my_channel_df['Date-Time'].loc[j] + timedelta(minutes=30)- start_of_week).total_seconds()/60
    for i in Movies for j in Time_slots
    )

model.addConstraint(
    start[i] <= x[i][j]*(my_channel_df['Date-Time'].loc[j] - start_of_week).total_seconds()/60 + (1 - x[i][j])*(2880 - movie_db_df['runtime_with_ads'].loc[i])
    for i in Movies for j in Time_slots
    )

print('Scheduling constaints added, ', time() - start_time)
# 5. Only one ad can be bought per avialable slot
model.addConstraint(
    xp.Sum(z0[i][r] for i in Movies) <= 1 for r in Ad_slots_0
)

model.addConstraint(
    xp.Sum(z1[i][s] for i in Movies) <= 1 for s in Ad_slots_1
)

model.addConstraint(
    xp.Sum(z2[i][t] for i in Movies) <= 1 for t in Ad_slots_2
)

# model.addConstraint(
#     xp.Sum(w[i][j] for i in Movies) + xp.Sum(v[i][j] for i in Movies) == 1 for j in Time_slots
# )


# 6. Only advertise movie if it is shown
model.addConstraint(
    z0[i][r] <= y[i] for i in Movies for r in Ad_slots_0
)
model.addConstraint(
    z1[i][s] <= y[i] for i in Movies for s in Ad_slots_1
)
model.addConstraint(
    z2[i][t] <= y[i] for i in Movies for t in Ad_slots_2
)
# model.addConstraint(
#     w[i][j] <= y[i] for i in Movies for j in Time_slots
# )

# # 7. Only sell ad slot for movie i at time j if it is shown then
# model.addConstraint(
#     v[i][j] <= x[i][j] for i in Movies for j in Time_slots
# )

# 8. Only advertise before the movie is scheduled
model.addConstraint(
    z0[i][r]*(conversion_rates_0_df['Date-Time'].loc[r] - start_of_week + timedelta(minutes=30)).total_seconds()/60 <= start[i]
    for i in Movies for r in Ad_slots_0
)
model.addConstraint(
    z1[i][s]*(conversion_rates_1_df['Date-Time'].loc[s] - start_of_week + timedelta(minutes=30)).total_seconds()/60 <= start[i]
    for i in Movies for s in Ad_slots_1
)
model.addConstraint(
    z2[i][t]*(conversion_rates_2_df['Date-Time'].loc[t] - start_of_week + timedelta(minutes=30)).total_seconds()/60 <= start[i]
    for i in Movies for t in Ad_slots_2
)
# model.addConstraint(
#     w[i][j]*(my_channel_df['Date-Time'].loc[j] - start_of_week + timedelta(minutes=30)).total_seconds()/60 <= start[i]
#     for i in Movies for j in Time_slots
# )

print('Advertising constraints added, ', time() - start_time)
# 9. The number per thousand of viewership is less than the viewership for the time slot
model.addConstraint(
    u[i]*viewership_units <=
    xp.Sum(
        movie_views_for_time_slot(x, i, j, movie_db_df, my_channel_df, Demos, population)
        for j in Time_slots
    )
    # + xp.Sum(
    #     own_advertised_views_for_time_slot(w, i , j, movie_db_df, my_channel_df, Demos, population)
    #     for j in Time_slots
    # )
    + xp.Sum(
        # comp_advertised_views_for_time_slot(z, i, j, c, movie_db_df, channel_dict, conversion_dict, Demos, Genres, population)
        comp_advertised_views_for_time_slot(z0, i, r, movie_db_df, channel_0_df, conversion_rates_0_df, Demos, Genres, population)
        for r in Ad_slots_0
    )
    + xp.Sum(
        comp_advertised_views_for_time_slot(z1, i, s, movie_db_df, channel_0_df, conversion_rates_0_df, Demos, Genres, population)
        for s in Ad_slots_1
    )
    + 
    xp.Sum(
        comp_advertised_views_for_time_slot(z2, i, t, movie_db_df, channel_0_df, conversion_rates_0_df, Demos, Genres, population)
        for t in Ad_slots_2
    )
    for i in Movies
)


# # 12. We only get contribution for viewership for movie i at time slot j if the time slot is sold
# model.addConstraint(
#     u[i] <= v[i][j]*(population/viewership_units)
#     for i in Movies for j in Time_slots
# )


print('Viewership constaints added, ', time() - start_time)

# 12. license fees and advertising slots bought must be within budget
# model.addConstraint(
#     xp.Sum(
#         y[i] * movie_db_df['license_fee'].iloc[i]
#         for i in Movies
#     )
#     + xp.Sum(
#         z[i][j][c] * calculate_ad_slot_price(j, channel_dict[c])
#         for i in Movies for j in Time_slots for c in Channels
#     )
#     <= budget
# )


##########################
# Objective Function
##########################

model.setObjective(
    xp.Sum(ad_sell_price_per_unit*u[i]  for i in Movies),
    sense=xp.maximize
)

print('time to intialise problem: ', time() - start_time)

# model.controls.maxtime = 300
# model.controls.maxnode = 1000
model.controls.miprelstop = 0.01
# model.controls.tunermaxtime = 1000
# model.controls.timelimit = 60
# model.tune('g')

solvestatus, solstatus = model.optimize()

now = datetime.now()
now = str(now).replace(" ", "_")
now = now.replace(":", "-")

# saved_sol_path = f'solutions/scheduling_advert_demos_{now}'
# model.write(saved_sol_path)

cost = sum(model.getSolution(y[i]) * movie_db_df['license_fee'].iloc[i] for i in Movies)
+ sum(model.getSolution(z0[i][j]) * calculate_ad_slot_price(j, channel_0_df) for i in Movies for j in Ad_slots_0)
+ sum(model.getSolution(z1[i][j]) * calculate_ad_slot_price(j, channel_1_df) for i in Movies for j in Ad_slots_1)
+ sum(model.getSolution(z2[i][j]) * calculate_ad_slot_price(j, channel_2_df) for i in Movies for j in Ad_slots_2)
print(cost)
# if solstatus != xp.SolStatus.INFEASIBLE or solstatus != xp.SolStatus.UNBOUNDED or solstatus != xp.SolStatus.UNBOUNDED:

output_dir = "./output"
# create the dir if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

with open(f"./output/output_{str(now)}.txt", "w") as f:
    f.write('Viewership: ')
    f.write(str(model.getObjVal))
    f.write('\n')
    for j in Time_slots:
        for i in Movies:
            if model.getSolution(x[i][j]) == 1:
                f.write("At ")
                f.write(str(my_channel_df['Date-Time'].loc[j]))
                f.write(" show movie ")
                f.write(movie_db_df['title'].loc[i])
                f.write('\n')
    # for j in Time_slots:
    #     for i in Movies:
    #         if model.getSolution(w[i][j]) == 1:
    #             f.write("At ")
    #             f.write(str(my_channel_df['Date-Time'].loc[j]))
    #             f.write(" on own channel advertise movie ")
    #             f.write(movie_db_df['title'].loc[i])
    #             f.write('\n')
    for j in Ad_slots_0:
        for i in Movies:
            if model.getSolution(z0[i][j]) == 1:
                f.write("At ")
                f.write(str(channel_0_df['Date-Time'].loc[j]))
                f.write(" on channel 0 advertise movie ")
                f.write(movie_db_df['title'].loc[i])
                f.write('\n')
    for j in Ad_slots_1:
        for i in Movies:
            if model.getSolution(z1[i][j]) == 1:
                f.write("At ")
                f.write(str(channel_1_df['Date-Time'].loc[j]))
                f.write(" on channel 1 advertise movie ")
                f.write(movie_db_df['title'].loc[i])
                f.write('\n')
    for j in Ad_slots_2:
        for i in Movies:
            if model.getSolution(z2[i][j]) == 1:
                f.write("At ")
                f.write(str(channel_2_df['Date-Time'].loc[j]))
                f.write(" on channel 2 advertise movie ")
                f.write(movie_db_df['title'].loc[i])
                f.write('\n')
f.close()
